First we create a large, sparse, positive definite matrix with condition number 1000.

In [ ]:
include("../FNC.jl") 
A = FNC.sprandsym(10000,0.002,1/1000);

Now we solve a linear system with a random right-hand side, without preconditioner.

In [ ]:
using IterativeSolvers
b = rand(10000)
minres(A,b,maxiter=100,tol=1e-10,log=true);
time_plain = @elapsed x,hist1 = cg(A,b,maxiter=400,tol=1e-10,log=true)

In [ ]:
using Plots
plot(hist1[:resnorm],label="", 
    title="CG with no preconditioning",
    xaxis=("iteration number"), yaxis=(:log10,"residual norm") )

For an SPD matrix we can use an incomplete Cholesky factorization. (It uses a lower triangular $\mathbf{L}=\mathbf{R}^T$ rather than an upper triangular $\mathbf{R}$.) However, because $A$ is a little close to singular (large condition number), the incomplete algorithm can perturb it enough to make it fail. So here we add a shift to the eigenvalues of $\mathbf{A}$ to make it more safely definite. 

In [ ]:
using Preconditioners,LinearAlgebra,SparseArrays

P = CholeskyPreconditioner(A+0.02I,5);

In [ ]:
time_prec = @elapsed x,hist2 = cg(A,b,Pl=P,maxiter=400,tol=1e-10,log=true)

In [ ]:
plot(hist1[:resnorm],label="no prec.", 
    xaxis=("iteration number"), yaxis=(:log10,"residual norm"),
    title="CG with incomplete Cholesky preconditioning")
plot!(hist2[:resnorm],label="iChol prec.")

You can see we made matters *worse* with this preconditioner, at least to judge by the residual. It's not always easy to get a good preconditioner!